In [10]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [11]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats,lngs)
lat_lngs

# Results are zipped
# The zip object packs each pair of lats and lngs having the same index in their 
# respective array into a tuple. If there are 1,500 latitudes and longitudes, 
# there will be 1,500 tuples of paired latitudes and longitudes, where each 
# latitude and longitude in a tuple can be accessed by the index of 0 and 1, respectively.

In [12]:
# Add the latitudes and longitudes to a list
# You can only unzip a zipped tuple once before it is removed from the computer's memory. 
# Make sure you unzip the latitudes and longitudes into the coordinates list before moving on.
coordinates = list(lat_lngs)

In [13]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [ ]:
# Use the tuple() furnction to display the latitude and longitude combination for coordinate in coordinates
for coordinate in coordinates:
    print(citipy.nearest_city(coordinate[0], coordinate[1]).city_name,
        citypi.nearest_city(coordinate[0], coordinate[1]).country_code)